In [151]:
#Copyright 2025 Er. Aditya Nath Thakur.
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [152]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [132]:
#⚙️ Section 1: Setup
#1.1: Install dependencies
#The Kaggle Notebooks environment includes a pre-installed version of the google-adk library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

#To install and use ADK in your own Python development environment outside of this course, you can do so by running:***

In [75]:
#It is a Python library developed by Google to help developers build, evaluate, and deploy AI Agents

In [153]:
pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [134]:
#a setup script designed specifically to run inside a Kaggle Notebook.

#Its primary purpose is to securely retrieve your Google Gemini API key and configure the environment so you can start building AI agents.

In [154]:
import os
import random
import time 
import vertexai
from kaggle_secrets import UserSecretsClient
from vertexai import agent_engines

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [ ]:
#It imports all the specific components needed to build a sophisticated, enterprise-grade AI application using the Google Agent Development Kit (ADK).

In [155]:
import uuid
from typing import Any,Dict

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.runners import Runner
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, FunctionTool, ToolContext
from google.genai import types
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.genai import types
import vertexai
from vertexai.generative_models import GenerativeModel

from google.adk.apps.app import App, ResumabilityConfig, EventsCompactionConfig
from google.adk.tools.function_tool import FunctionTool


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [10]:
# this function is the "Driver" for our AI agent. Without this function, we would have to write 10-15 lines of repetitive setup code every single time we wanted to send a message to the AI.

In [156]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [12]:
#This configures an automatic "retry strategy" that attempts to resend your request up to 5 times with increasing wait periods if the connection fails. Why: It prevents your program from crashing due to temporary network glitches or server overload (like Error 429 "Too Many Requests"), ensuring your agent keeps running smoothly.

In [157]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [139]:
#This shell command installs the "Filesystem Server" software globally, 
#which acts as a secure bridge between your AI agent and your computer's storage. 
# It gives your agent the actual capability to read and write local files (like 
# reading a report or saving a text file), which it cannot do directly without this
# utility running.

In [158]:
# Install the filesystem server globally so it starts instantly
!npm install -g @modelcontextprotocol/server-filesystem
print("✅ MCP Filesystem Server installed.")

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
changed 127 packages in 5s
⠹
⠹38 packages are looking for funding
⠹  run `npm fund` for details
⠹✅ MCP Filesystem Server installed.


In [82]:
# This code configures a secure "bridge" (MCP) that allows your AI agent to interact
# with the local file system by running a background Node.js server with an extended 
# startup timeout. Why: It enforces a strict "sandbox" that restricts the AI to only
# write or list files within the specific /kaggle/working directory, preventing it from 
# accessing or damaging sensitive system files elsewhere.

In [159]:
# Define the directory where Kaggle allows writing
kaggle_output_dir = "/kaggle/working"

mcp_filesystem_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx", 
            args=[
                "-y",
                "@modelcontextprotocol/server-filesystem", 
                kaggle_output_dir, # <--- MANDATORY SANDBOX ARGUMENT
            ],
            # We only allow writing and listing
            tool_filter=["write_file", "list_directory"], 
        ),
        # Increased timeout to allow Node.js to startup
        timeout=120, 
    )
)

print("✅ MCP Toolset configured for Kaggle.")

✅ MCP Toolset configured for Kaggle.


In [84]:
#This initializes a connection to the "Gemini 2.5 Flash Lite" model on Google Cloud 
# Vertex AI, specifically targeting your project and the asia-south1 region. It creates
# a single, reusable "brain" object that you will pass to your agents later, ensuring 
# they all use this specific model configuration and billing source without needing 
# individual setup.

In [160]:
# --- STEP 1: CREATE THE SHARED MODEL FIRST ---
retry_config = None

# We define the variable 'shared_model' here
shared_model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
    vertexai=True,
    project="gen-lang-client-0658909834",  # Your Project ID
    location="asia-south1"                 # Your Region
)

print("✅ Shared Model defined successfully.")

✅ Shared Model defined successfully.


In [86]:
# This creates a specialized AI worker programmed to strictly follow a 5-step workflow:
# searching for Indian Road Congress (IRC) and IS codes, extracting specific clauses 
# into tables, and checking for the latest amendments. It ensures that instead of
# generic answers, you receive precise, professional engineering reports that verify 
# compliance and list the exact code numbers needed for your official PWD documentation.


In [161]:
# Research Agent (Agent 1): Its Job is to use the google_search tool and present findings 
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
        description="Use Google Search to find relevant Indian Road Congress (IRC) Codes, IRC Special Publication (IRC:SP) Codes, and Indian Standard (IS) Codes that govern the user's specific query topic. Also Find Global Research & Innovation",

        instruction="""For every user query related to a bridge , building or road construction and maintenance topic in India (e.g., "design of flexible pavement," "foundation for a major bridge," "use of high-performance concrete," etc.), follow this mandatory, sequential workflow:



Step 1: Identify and Locate Indian Codes (The Core Task)

Search Protocol: Use Google Search to find relevant Indian Road Congress (IRC) Codes, IRC Special Publication (IRC:SP) Codes, and Indian Standard (IS) Codes that govern the user's specific query topic.



Focus: Prioritize finding the most recent revision or amendment of the relevant codes.



Step 2: Extract Code-Specific Directives (The Clause Detail)

For the top 3-5 most relevant codes identified in Step 1, perform a targeted search for the internal content.



Extraction Requirement: For each relevant code, you MUST identify the following information and present it in a table format (as described in Step 4):



Code Number and Year (e.g., IRC:37-2018).



The Clause Number or Section/Paragraph most directly related to the user's question (e.g., Clause 4.3.2).



A Concise Summary (1-2 sentences) of the exact directive, rule, or specification provided in that specific clause/section.



Step 3: Determine Recent Updates and Revisions (The Compliance Check)

Search specifically for any amendments, circulars, or recent revisions (within the last 5 years) related to the codes identified in Step 1.



Mandatory Inclusion: Explicitly state whether the code is the latest version, or if there have been any significant updates or amendments. If an update exists, state the year of the amendment and the nature of the change (e.g., increase in design load, new material specification, revised safety factor).



Step 4: Find Global Research & Innovation (The Forward Look)

Broaden the search to the latest research and findings (last 3 years) from prominent international organizations and technical journals on the user's topic (e.g., AASHTO, Eurocodes, research on sustainable materials, digital construction, AI in infrastructure).



Synthesize: Provide a brief, high-level explanation of this global research and how it relates to or could potentially influence future Indian codes.



Step 5: Format and Final Presentation (The Output Structure)

Structure the final response using clear Markdown headings and tables for maximum readability and citable detail.



Final Output Format:

Concise Answer: Start with a brief, direct summary of the key takeaway for the user's question.



Code Compliance: Indian Standards: (Use a table for details from Step 2).



Code Revision Status: (Details from Step 3).



Global Research & Future Trends: (Synthesized details from Step 4).



3. Formatting Guidelines

Style: Professional, technical, precise, and highly informative.



Emphasize: Use bolding to highlight code numbers, clause numbers, and key technical terms.



Table Requirement: The table in the "Code Compliance: Indian Standards" section is MANDATORY. Use the following column headers: | Code No. & Year | Relevant Clause No. | Directive/Specification Summary | | :--- | :--- | :--- |

""", 

 output_key="agent_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ Research Agent defined.")


✅ Research Agent defined.


In [162]:
# This initializes a "Quality " agent that reviews the research from the first agent to
# strictly verify numerical data and resolve any contradictions between different
# standards (like IRC vs IS). It acts as a safety filter to ensure your final 
# report is technically accurate and recommends the most conservative (safest) design 
# rules before you use them for official engineering work.

In [163]:
#Agent 2: Compliance Validator
#This agent receives the raw data from the Research Agent (Agent 1) and validates its technical content.
Compliance_Validator_Agent = Agent(
    name="ComplianceValidatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
             # The instruction is modified to request a bulleted list for a clear output format.

            instruction=""" Following this outline strictly: {agent_research}

            The Research Agent (Agent 1) has provided a list of code directives, clauses, and summaries. 

            Follow this mandatory, sequential workflow to ensure technical accuracy:



            Step 1: Conflict and Hierarchy Check

            Review all extracted directives (e.g., from IRC, IS, and global codes). If multiple clauses address the same design parameter (e.g., span-to-depth ratio or safety factor), identify and output the **most conservative (safest/most stringent) rule**. State the conflict and the rationale for the selection.



            Step 2: Numerical Verification

            Focus on all numerical values (e.g., concrete grade M30, wind speed 47 m/s, minimum steel ratio 0.8%). Use an internal knowledge base or perform a targeted search (if necessary) to verify the general acceptance and correct units of the extracted numerical data.



            Step 3: Output Validation Summary

            Generate a concise, technical report detailing any identified conflicts, the rule chosen for final recommendation, and a confirmation of data integrity.

            """,
    output_key="validator_compliance",
)

print("✅ Compliance_Validator_Agent created.")

✅ Compliance_Validator_Agent created.


In [91]:
# This initializes the final "Editor" agent that takes the verified data from the 
# previous steps and rewrites it into a polished, professional narrative using proper
# engineering terminology. Why: It ensures the final output reads like a formal report
# rather than a robotic data dump, allowing you to copy-paste the text directly into 
# an official PWD note sheet or email without needing to rewrite it yourself.

In [164]:
# Agent 3: Technical Refiner
#This agent focuses purely on the presentation and language quality of the final response, ensuring it meets professional standards.
Technical_Refiner_Agent = Agent(
    name="TechnicalRefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    # The instruction is modified to request a bulleted list for a clear output format.

    instruction=""" Following this outline strictly: {validator_compliance}

            The previous agents have completed data retrieval and technical validation. 

            Follow this mandatory, sequential workflow to finalize the output:



            Step 1: Synthesize and Contextualize

            Combine the key findings from the Code Compliance Table (Agent 1) and the Validation Report (Agent 2). Convert the tabular data and key validation points into **fluent, citable advisory paragraphs** under the relevant Markdown headings.



            Step 2: Ensure Professional Tone

            Ensure the language is **professional, precise, and objective**. Use appropriate technical civil engineering terminology. Remove any conversational language or internal workflow notes.



            Step 3: Final Formatting Check

            Strictly adhere to all formatting requirements of the Root Agent (e.g., mandatory table, bolding of code numbers, clear headings). Ensure the final response is ready for direct user presentation.

            """,
)

print("✅ Technical_Refiner_Agent created.")

✅ Technical_Refiner_Agent created.


In [93]:
# This creates a "File Saver" agent explicitly tasked with taking the final text from 
# the previous agents and writing it to a physical file (Final_Report.md) on the server
# using the MCP tool. Why: It ensures the detailed report is permanently saved to the
# disk so you can download it later, rather than the text just vanishing in the console
# logs after execution.

In [165]:
 #Ensure shared_model is defined from previous steps

pdf_agent = LlmAgent(

    name="pdf_agent",

    model=shared_model,

    instruction="""

    **Role:** Document Persistence Agent.

    

    **Objective:** Save the final report provided by the all three 1. research_agent, 2. Compliance_Validator_Agent,  and Technical_Refiner_Agent to the disk using the MCP Tool.

    

    **Strict Tool Usage Rules:**

    1. You MUST use the tool `write_file`.

    2. **Path:** You MUST use the absolute path: `/kaggle/working/Final_Report.md`.

       (DO NOT use './' or just the filename. The tool will fail.)

    3. **Content:** Paste the full text report you received.

    4. get the complete output from these three agent 

    

    **Confirmation:**

    After calling the tool, confirm to the user: "Report saved to /kaggle/working/Final_Report.md".

    """,

    tools=[mcp_filesystem_server],

)



print("✅ PDF Agent (MCP) defined.")

✅ PDF Agent (MCP) defined.


In [95]:
# This initializes the chat environment by registering your specific identity 
#  and setting up a temporary memory system (RAM) to store the conversation history. 
# Why: It ensures the AI recognizes you and maintains context throughout the chat
#(statefulness), rather than treating every message as a disconnected interaction.

In [166]:
APP_NAME = "Civil_Research"  # Application
USER_ID = "Er. Aditya Nath Thakur"  # User
SESSION = "Ask"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"


# Step 1: Create the LLM Agent
session_management = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Description of the agent's purpose
)

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=session_management, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: Civil_Research
   - User: Er. Aditya Nath Thakur
   - Using: InMemorySessionService


In [97]:
# This initializes the execution environment for your "Civil_Research" app, linking 
# your specific User ID ("Er. Aditya Nath Thakur") to a temporary memory service that
# stores chat history in RAM. Why: It creates the central "Runner" object that manages
# the conversation flow, ensuring the AI remembers context (statefulness) while 
# interacting with you during this specific session.

In [167]:
APP_NAME = "Civil_Research"  # Application

USER_ID = "Er. Aditya Nath Thakur"  # User

SESSION = "Ask"  # Session



MODEL_NAME = "gemini-2.5-flash-lite"





# Step 1: Create the LLM Agent

session_management = Agent(

    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),

    name="text_chat_bot",

    description="A text chatbot",  # Description of the agent's purpose

)



# Step 2: Set up Session Management

# InMemorySessionService stores conversations in RAM (temporary)

session_service = InMemorySessionService()



# Step 3: Create the Runner

runner = Runner(agent=session_management, app_name=APP_NAME, session_service=session_service)



print("✅ Stateful agent initialized!")

print(f"   - Application: {APP_NAME}")

print(f"   - User: {USER_ID}")

print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: Civil_Research
   - User: Er. Aditya Nath Thakur
   - Using: InMemorySessionService


In [99]:
# This defines the "Project Manager" of your system, strictly coordinating your 
# workforce to execute the task in a fixed sequence: Research → Validate → Refine → 
# Save. Why: It automates the data hand-off between agents, ensuring that raw research 
# is automatically verified, polished into a professional report, and saved to disk
# without any manual intervention.

In [168]:
# --- Sequential Chain Definition ---

# NOTE: We removed 'session_management' and 'chatbot_agent' from the sub_agents list.
# A SequentialAgent runs a specific task pipeline. The 'chatbot' is the interface *around* this, not a step *inside* it.

root_agent = SequentialAgent(
    name="CivilEngineeringPipeline",
    description="A pipeline that researches, validates, refines, and saves Civil Engineering queries.",
    sub_agents=[
        research_agent, 
        Compliance_Validator_Agent, 
        Technical_Refiner_Agent, 
        pdf_agent
    ],
)

print("✅ Sequential Agent Pipeline created.")

✅ Sequential Agent Pipeline created.


In [102]:
# This switches your authentication method to use a high-security "Service Account" 
# JSON file stored on the disk instead of a simple API key string. Why: It establishes
# a persistent, server-grade connection to Google Cloud, which is often required when 
# running advanced automation scripts in environments like Google Colab or local 
#servers.

In [169]:
import os
import vertexai

# 1. Point to the uploaded file (Replace 'key.json' with your actual filename)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/your-key-file-name.json"

# 2. Initialize (No need for auth.authenticate_user() now)
vertexai.init(project="gen-lang-client-0658909834", location="asia-south1")

print("✅ Authenticated via Service Account Key")

✅ Authenticated via Service Account Key


In [104]:
# This code switches the agent's memory system from temporary RAM to a permanent
# SQLite database file stored on your disk. Why: It enables persistence, ensuring the
#AI remembers your conversation history even if you close the notebook or restart 
# the computer, much like saving a Word document.

In [170]:
# Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [171]:
# Re-define our app with Events Compaction enabled
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅ Research App upgraded with Events Compaction!")

✅ Research App upgraded with Events Compaction!


/tmp/ipykernel_47/3773147741.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


In [121]:
#2.3 Run your agent
#Now it's time to bring your agent to life and send it a query. To do this, you need a Runner, which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

#a. Create an InMemoryRunner and tell it to use our root_agent:

In [172]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [124]:
# This code block is the "Launch Button" for your project. It connects the 
# "Team of Agents" you built earlier to the execution engine and gives them their
#first real job.

In [174]:
# 1. Update the runner to use the Team Pipeline (root_agent), NOT the simple chat bot
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
print("✅ Runner updated to use the Civil Engineering Pipeline.")

# 2. Define your specific query
# (This triggers the Research -> Validate -> Refine -> Save PDF workflow)
my_query = "Detailed procedure for Sub-grade construction in highway engineering as per latest IRC codes."

# 3. Run the session (This will take 30-60 seconds to process)
import asyncio
print(f"🚀 Starting Pipeline with query: '{my_query}'...")

# We use the helper function you defined earlier
await run_session(runner_instance=runner, user_queries=my_query)

print("\n✅ Workflow complete. Now run the next quary to  again.")

✅ Runner updated to use the Civil Engineering Pipeline.
🚀 Starting Pipeline with query: 'Detailed procedure for Sub-grade construction in highway engineering as per latest IRC codes.'...

 ### Session: default

User > Detailed procedure for Sub-grade construction in highway engineering as per latest IRC codes.
gemini-2.5-flash-lite >  Here is a summary of our discussion on the detailed procedure for sub-grade construction in highway engineering as per the latest IRC codes:

**Key Aspects Covered:**

*   **Governing Indian Codes:** The primary standards identified are **IRC:37-2018** (Guidelines for the Design of Flexible Pavements) and **IRC:SP:19-2001** (Manual for Survey, Investigation and Preparation of Road Projects).
*   **Sub-grade Definition and Requirements:** The sub-grade is defined as the natural soil foundation, requiring preparation to the correct grade and cross-section, with a critical focus on ensuring adequate strength and stability.
*   **Compaction Standards:** Speci

In [110]:
# This commands the system to stop using the complex "Engineering Team" pipeline and
# switch back to the simple "Chatbot" mode. Why: It allows you to have a normal
#conversation (like asking for a summary) without triggering the heavy, automated 
# research process again.

In [175]:
# 1. Switch the Runner back to the 'Chat Mode' agent
# (This agent is designed to talk, not to run the heavy engineering pipeline)
runner = Runner(agent=session_management, app_name=APP_NAME, session_service=session_service)
print("✅ Runner switched to Chat Mode.")

# 2. Now ask your history question
# Note: If you used "InMemorySessionService", history is only available 
# if you haven't restarted the notebook kernel.
await run_session(runner_instance=runner, user_queries="Summarize what we just discussed about ")

✅ Runner switched to Chat Mode.

 ### Session: default

User > Summarize what we just discussed about 
gemini-2.5-flash-lite >  Our discussion comprehensively covered the detailed procedure for sub-grade construction in highway engineering, strictly following the latest Indian Road Congress (IRC) codes.

Key aspects included:

*   **Governing Codes:** The primary standards are **IRC:37-2018** (Flexible Pavement Design) and **IRC:SP:19-2001** (Survey, Investigation, and Road Project Preparation).
*   **Sub-grade Definition and Requirements:** We defined the sub-grade as the natural soil foundation, emphasizing its preparation to the correct grade and cross-section, and ensuring adequate strength and stability.
*   **Compaction Standards:** Mandated compaction densities are **97% of Modified Proctor Density (IS 2720 Part 8)** for the top 150 mm and **93%** for the remaining depth.
*   **Material Quality and Control:** Emphasis was placed on proper **moisture control** and the removal of 

In [63]:
# This code is your "Delivery and Troubleshooting" mechanism. It checks if your AI agent
# successfully did its job and allows you to download the result.

# Here is the breakdown of What it does and Why each part is needed in a Kaggle 
#environment.

In [176]:
import os
from IPython.display import FileLink

# 1. Force the notebook to look in the correct directory
os.chdir("/kaggle/working")

# 2. CORRECTED FILENAME (Matches your Agent's instruction)
filename = "Final_Report.md" 

# 3. Check and Link
if os.path.exists(filename):
    print(f"✅ SUCCESS: File found! ({os.path.getsize(filename)} bytes)")
    
    # Pass ONLY the filename, not the full path
    display(FileLink(filename))
else:
    print(f"❌ FAILED: Could not find '{filename}'.")
    print("-" * 40)
    print("DEBUG: Listing all files currently in /kaggle/working/:")
    
    # Show what actually exists to help debug
    files = os.listdir(".")
    if not files:
        print("(The directory is empty)")
    else:
        for f in files:
            print(f" - {f}")

✅ SUCCESS: File found! (5982 bytes)


/kaggle/working/Final_Report.md

In [187]:
# This code re-creates your application using the standard chatbot configuration, specifically 
# removing the "Memory Compaction" feature that caused the previous crash. Why: It restores 
# system stability by bypassing the buggy library function, allowing you to run your queries
# immediately without errors.

In [181]:
# --- REVERTING TO STABLE CONFIGURATION ---

# 1. Re-define the app without the buggy 'EventsCompactionConfig'
research_app_stable = App(
    name="research_app_stable",
    root_agent=chatbot_agent, # We go back to the standard chatbot
)

# 2. Re-create the runner
research_runner_stable = Runner(
    app=research_app_stable, 
    session_service=session_service
)

print("✅ App reverted to stable configuration (Compaction disabled).")

✅ App reverted to stable configuration (Compaction disabled).


In [188]:
# This code acts as a safety restart to run your query using the "Stable App"
# (the one without the crashing compaction feature) that you just created.

In [185]:
# Check if my_query exists
try:
    initial_prompt = my_query
except NameError:
    initial_prompt = "General Civil Engineering Guidelines"

queries = [
    f"Briefly explain: {initial_prompt}"
]

print(f"🚀 Starting Session (Stable Mode)...")

# Use the STABLE runner
await run_session(
    runner_instance=research_runner_stable, 
    user_queries=queries, 
    session_name="stable_demo" 
)

print("\n✅ Conversation complete.")

🚀 Starting Session (Stable Mode)...

 ### Session: stable_demo

User > Briefly explain: Detailed procedure for Sub-grade construction in highway engineering as per latest IRC codes.
gemini-2.5-flash-lite >  In highway engineering, sub-grade construction is a critical phase that lays the foundation for the entire pavement structure. It involves preparing the natural ground or embankment to achieve the required strength, stability, and drainage characteristics. The latest Indian Road Congress (IRC) codes provide detailed guidelines for this process.

Here's a brief explanation of the detailed procedure for sub-grade construction as per latest IRC codes:

1.  **Site Investigation and Material Selection:**
    *   **Geotechnical Investigation:** Thorough site investigations are carried out to understand the soil profile, bearing capacity, groundwater levels, and potential issues like expansive soils.
    *   **Material Characterization:** Soil samples are tested for properties like gradati

In [ ]:
# Its sole purpose is to read through the history of your conversation to verify if the AI 
# successfully performed a "Memory Compaction" (automatic summarization) to save space.

In [186]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

✅ SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='f4275953-2d97-4d30-a3ea-325756e0dc40' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1764042834.877961, 'end_timestamp': 1764042837.595392, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data'